In [0]:
import os
import pandas as pd
from google.colab import auth
from datetime import datetime
auth.authenticate_user()
!gcloud source repos clone github_aistream-peelout_flow-forecast --project=gmap-997
os.chdir('/content/github_aistream-peelout_flow-forecast')
!git checkout -t origin/covid_fixes
!python setup.py develop
!pip install -r requirements.txt
!mkdir data
from flood_forecast.trainer import train_function
!pip install git+https://github.com/CoronaWhy/task-geo.git
!wandb login

$ git clone https://github.com/AIStream-Peelout/flow-forecast
Cloning into '/content/github_aistream-peelout_flow-forecast'...
remote: Total 3703 (delta 2373), reused 3703 (delta 2373)
Receiving objects: 100% (3703/3703), 2.65 MiB | 11.13 MiB/s, done.
Resolving deltas: 100% (2373/2373), done.
Project [gmap-997] repository [github_aistream-peelout_flow-forecast] was cloned to [/content/github_aistream-peelout_flow-forecast].


To take a quick anonymous survey, run:
  $ gcloud survey

Branch 'covid_fixes' set up to track remote branch 'covid_fixes' from 'origin'.
Switched to a new branch 'covid_fixes'
/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '0.01dev' to '0.1.dev0'
  warnings.warn(tmpl.format(**locals()))
running develop
running egg_info
creating flood_forecast.egg-info
writing flood_forecast.egg-info/PKG-INFO
writing dependency_links to flood_forecast.egg-info/dependency_links.txt
writing requirements to flood_forecast.egg-info/requires.txt

In [0]:
def make_config_file(file_path, df_len):
  run = wandb.init(project="covid-forecast")
  wandb_config = wandb.config
  train_number = df_len * .7
  validation_number = df_len *.9
  config_default={                 
    "model_name": "MultiAttnHeadSimple",
    "model_type": "PyTorch",
    "model_params": {
      "number_time_series":3,
      "seq_len":wandb_config["forecast_history"], 
      "output_seq_len":wandb_config["out_seq_length"],
      "forecast_length":wandb_config["out_seq_length"]
     },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"],
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": int(train_number),
       "valid_start":int(train_number+1),
       "valid_end": int(validation_number),
       "target_col": ["new_cases"],
       "relevant_cols": ["new_cases", "month", "weekday"],
       "scaler": "StandardScaler", 
       "interpolate": False
    },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "Adam",
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    
    },
    "GCS": False,
    
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE"],
   "inference_params":
   {     
         "datetime_start":"2020-04-21",
          "hours_to_forecast":10, 
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"],
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": ["new_cases", "month", "weekday"],
             "target_col": ["new_cases"],
             "scaling": "StandardScaler",
             "interpolate_param": False
          }
      }
  }
  wandb.config.update(config_default)
  return config_default

sweep_config = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "batch_size": {
            "values": [2, 3, 4, 5]
        },
        "lr":{
            "values":[0.001, 0.002, 0.004, 0.01]
        },
        "forecast_history":{
            "values":[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        },
        "out_seq_length":{
            "values":[1, 2, 3]
        }
    }
}

In [0]:
def format_corona_data(region_df:pd.DataFrame, region_name:str):
  """
  Format data for a specific region into 
  a format that can be used with flow forecast. 
  """
  if region_name == 'county':
    region_name = region_df['full_county'].iloc[0]
  else:
    region_name = region_df['state'].iloc[0]
  #else:
    #region_name = region_df['country'].iloc[0]
  print(region_name)
  region_df['datetime'] = region_df['date']
  region_df['precip'] = 0
  region_df['temp'] = 0
  region_df = region_df.fillna(0)
  region_df['new_cases'] = region_df['cases'].diff()
  region_df.iloc[0]['new_cases'] = 0
  region_df= region_df.fillna(0)
  region_df.to_csv(region_name+".csv")
  return region_df, len(region_df), region_name+".csv"

def loop_through_geo_codes(df, column='full_county'):
  df_county_list = []
  df['full_county'] = df['state'] + "_" + df['county'] 
  for code in df['full_county'].unique():
    mask = df['full_county'] == code
    df_code = df[mask]
    ts_count = len(df_code)
    if ts_count > 60:
      df_county_list.append(df_code)
  return df_county_list 

def fetch_time_series() -> pd.DataFrame:
    """Fetch raw time series data from coronadatascraper.com
    Returns:
        pd.DataFrame: raw timeseries data at county/sub-region level
    """
    if 1==1:
        url = "https://coronadatascraper.com/timeseries.csv"
        urllib.request.urlretrieve(url, "timeseries.csv")

    time_series_df = pd.read_csv("timeseries.csv")
    return time_series_df

In [0]:
import urllib
df = fetch_time_series()
df['month'] = pd.to_datetime(df['date']).map(lambda x: x.month)
df['weekday'] = pd.to_datetime(df['date']).map(lambda x: x.weekday())
df_list = loop_through_geo_codes(df)
region_df, full_len, file_path = format_corona_data(df_list[9], 'county')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Washington_Snohomish County


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Run sweep

In [0]:
import wandb
sweep_id = wandb.sweep(sweep_config, project="covid-forecast")
wandb.agent(sweep_id, lambda:train_function("PyTorch", make_config_file(file_path, full_len)))


In [0]:
region_df

,name,level,city,county,state,country,population,lat,long,url,aggregate,tz,cases,deaths,recovered,active,tested,hospitalized,discharged,icu,growthFactor,date,full_county,datetime,precip,temp,new_cases
0,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01-22,Flanders_Antwerp,2020-01-22,0,0,0.0
1,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01-23,Flanders_Antwerp,2020-01-23,0,0,0.0
2,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01-24,Flanders_Antwerp,2020-01-24,0,0,0.0
3,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01-25,Flanders_Antwerp,2020-01-25,0,0,0.0
4,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01-26,Flanders_Antwerp,2020-01-26,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,6211.0,0.0,0.0,0.0,0.0,2244.0,1972.0,0.0,0.0,2020-04-29,Flanders_Antwerp,2020-04-29,0,0,117.0
99,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,6284.0,0.0,0.0,0.0,0.0,2266.0,2021.0,0.0,0.0,2020-04-30,Flanders_Antwerp,2020-04-30,0,0,73.0
100,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,6302.0,0.0,0.0,0.0,0.0,2280.0,2074.0,0.0,0.0,2020-05-01,Flanders_Antwerp,2020-05-01,0,0,18.0
101,"Antwerp, Flanders, Belgium",county,0,Antwerp,Flanders,Belgium,1847486.0,51.2485,4.7175,https://epistat.wiv-isp.be/,0,Europe/Brussels,6302.0,0.0,0.0,0.0,0.0,2298.0,2096.0,0.0,0.0,2020-05-02,Flanders_Antwerp,2020-05-02,0,0,0.0


In [0]:
import matplotlib.pyplot as plt

In [0]:
plt.show()